In [1]:
import numpy as np
import mindspore as ms
import mindspore.dataset as ds
import mindspore.ops as ops
from mindspore import Tensor, grad
from d2l import mindspore as d2l

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [2]:
class SyntheticData():
    def __init__(self):
        self.features, self.labels = d2l.synthetic_data(true_w, true_b, 1000)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
    def __len__(self):
        return len(self.labels)

def load_array(data_arrays, column_names, batch_size, is_train=True):  
    """构造一个MindSpore数据迭代器。"""
    dataset = ds.GeneratorDataset(data_arrays, column_names, shuffle=is_train)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = SyntheticData()
dataset = load_array(data_iter, ['features', 'labels'], batch_size)

In [3]:
next(iter(dataset))

[Tensor(shape=[10, 2], dtype=Float32, value=
 [[ 5.01442850e-01, -5.36665499e-01],
  [-4.36666548e-01, -7.02304423e-01],
  [ 5.77616572e-01,  1.03502202e+00],
  ...
  [ 1.56653082e+00, -3.97515804e-01],
  [-1.91809312e-02,  5.47854453e-02],
  [ 1.47573948e-01,  1.43204105e+00]]),
 Tensor(shape=[10, 1], dtype=Float32, value=
 [[ 7.03418970e+00],
  [ 5.72131443e+00],
  [ 1.83355534e+00],
  ...
  [ 8.66133785e+00],
  [ 3.96633053e+00],
  [-3.58127326e-01]])]

In [4]:
# nn是神经网络的缩写
from mindspore import nn
from mindspore.common.initializer import initializer, Normal

net = nn.SequentialCell([nn.Dense(2, 1)])

In [5]:
net[0].weight = initializer(Normal(), net[0].weight.shape, ms.float32)
net[0].bias = initializer('zero', net[0].bias.shape, ms.float32)

In [6]:
loss = nn.MSELoss()

In [7]:
optimizer = nn.SGD(net.trainable_params(), learning_rate=0.03)

In [9]:
net_with_loss = nn.WithLossCell(net, loss)
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in dataset:
        l = net_with_loss(X, y)
        grad_fn = grad(net_with_loss, grad_position=None, weights=optimizer.parameters)
        grads = grad_fn(X, y)
        loss = ops.depend(l, optimizer(grads))
    l = net_with_loss(ms.Tensor(data_iter.features), ms.Tensor(data_iter.labels))
    print(f'epoch {epoch + 1}, loss {l.asnumpy():f}')

epoch 1, loss 0.000223
epoch 2, loss 0.000102
epoch 3, loss 0.000102


In [10]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： [Tensor(shape=[], dtype=Float32, value= -0.000443697)
 Tensor(shape=[], dtype=Float32, value= 0.000309467)]
b的估计误差： [0.00081301]
